In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [2]:
def create_improved_model(input_dim):
    """
    Crear un modelo mejorado
    """
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim,
              kernel_initializer='glorot_uniform'),
        Dropout(0.3),
        Dense(32, activation='relu',
              kernel_initializer='glorot_uniform'),
        Dropout(0.3),
        Dense(1, activation='sigmoid',
              kernel_initializer='glorot_uniform')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [17]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, class_weights, model_name="Modelo"):
    """
    Entrenar y evaluar modelo
    """
    print(f"Entrenando {model_name}")
    
    # Callbacks para monitorear el entrenamiento
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=15, 
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.2, 
        patience=10, 
        min_lr=0.0001,
        verbose=1
    )
    
    # Entrenar modelo
    history = model.fit(
        X_train, y_train,
        class_weight=class_weights, 
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=32,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    # Evaluar modelo
    print(f"\nEvluación {model_name}")
    
    # Predicciones
    y_pred_prob = model.predict(X_test, verbose=0)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    
    # Estadísticas de probabilidades
    print(f"Estadísticas de probabilidades:")
    print(f"  Min: {y_pred_prob.min():.6f}")
    print(f"  Max: {y_pred_prob.max():.6f}")
    print(f"  Mean: {y_pred_prob.mean():.6f}")
    print(f"  Std: {y_pred_prob.std():.6f}")
    print(f"  Mediana: {np.median(y_pred_prob):.6f}")
    
    # Distribución de predicciones
    print(f"\nDistribución de predicciones:")
    unique, counts = np.unique(y_pred, return_counts=True)
    for val, count in zip(unique, counts):
        label = "exitoso" if val == 1 else "no_exitoso"
        print(f"  {label}: {count}")
    
    # Matriz de confusión
    print(f"\nMatriz de confusión:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    
    # Reporte de clasificación
    print(f"\nReporte de clasificación:")
    print(classification_report(y_test, y_pred, target_names=['no_exitoso', 'exitoso']))
    
    # Verificar pesos del modelo
    print(f"\nAnálisis de pesos:")
    for i, layer in enumerate(model.layers):
        if hasattr(layer, 'get_weights') and layer.get_weights():
            weights = layer.get_weights()[0]
            print(f"  Capa {i}: shape={weights.shape}, rango=[{weights.min():.6f}, {weights.max():.6f}]")
    
    return history, y_pred_prob

In [16]:
# Cargar datos
df = pd.read_csv("C:/Users/sofia/Downloads/Repositorios analítica/analítica/proyecto2/Datos/Datos2011Limipios.csv")
print(f"Datos originales: {df.shape}")
print(f"Distribución original:\n{df['resultado'].value_counts()}")

Datos originales: (494094, 29)
Distribución original:
resultado
no_exitoso    476685
exitoso        17409
Name: count, dtype: int64


In [5]:
categorical_columns = ['cole_depto_ubicacion', 'cole_jornada', 'cole_naturaleza', 
                       'fami_educacionmadre', 'fami_educacionpadre', 'fami_estratovivienda']

In [6]:
existing_categorical = [col for col in categorical_columns if col in df.columns]
print(f"Columnas categóricas disponibles: {existing_categorical}")

Columnas categóricas disponibles: ['cole_depto_ubicacion', 'cole_jornada', 'cole_naturaleza', 'fami_educacionmadre', 'fami_educacionpadre', 'fami_estratovivienda']


In [7]:
df_processed = df.copy()

In [8]:
encoders = {}
for col in existing_categorical:
    print(f"Procesando {col}...")
    # Llenar valores nulos
    df_processed[col] = df_processed[col].fillna('unknown')
    
    # Codificar con LabelEncoder
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col].astype(str))
    encoders[col] = le
    
    print(f"  {col}: {len(le.classes_)} clases únicas")

Procesando cole_depto_ubicacion...
  cole_depto_ubicacion: 34 clases únicas
Procesando cole_jornada...
  cole_jornada: 6 clases únicas
Procesando cole_naturaleza...
  cole_naturaleza: 2 clases únicas
Procesando fami_educacionmadre...
  fami_educacionmadre: 11 clases únicas
Procesando fami_educacionpadre...
  fami_educacionpadre: 11 clases únicas
Procesando fami_estratovivienda...
  fami_estratovivienda: 6 clases únicas


In [9]:
feature_columns = existing_categorical
X = df_processed[feature_columns].copy()
y = df_processed['resultado'].map({'no_exitoso': 0, 'exitoso': 1})

print(f"\nFeatures finales: {X.shape}")
print(f"Columnas: {list(X.columns)}")


Features finales: (494094, 6)
Columnas: ['cole_depto_ubicacion', 'cole_jornada', 'cole_naturaleza', 'fami_educacionmadre', 'fami_educacionpadre', 'fami_estratovivienda']


In [11]:
from sklearn.utils import class_weight
# Asignación de pesos a las clases para evitar el sesgo
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights = {cls: weight for cls, weight in zip(np.unique(y), weights)}
class_weights

{np.int64(0): np.float64(0.5182604864847855),
 np.int64(1): np.float64(14.190763398242288)}

In [12]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X), 
    columns=X.columns, 
    index=X.index
)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nConjuntos de entrenamiento y prueba:")
print(f"  Train: {X_train.shape}, distribución: {np.bincount(y_train)}")
print(f"  Test: {X_test.shape}, distribución: {np.bincount(y_test)}")


Conjuntos de entrenamiento y prueba:
  Train: (395275, 6), distribución: [381348  13927]
  Test: (98819, 6), distribución: [95337  3482]


In [18]:
improved_model = create_improved_model(X_train.shape[1])
improved_history, improved_probs = train_and_evaluate_model(
    improved_model, X_train, y_train, X_test, y_test, class_weights, "Mejorado"
)
    
# Guardar el mejor modelo
best_model = improved_model
best_probs = improved_probs

c:\Users\sofia\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Entrenando Mejorado
Epoch 1/50
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.8421 - loss: 0.3851 - val_accuracy: 0.8635 - val_loss: 0.3210 - learning_rate: 0.0010
Epoch 2/50
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 36s 3ms/step - accuracy: 0.8571 - loss: 0.3607 - val_accuracy: 0.8643 - val_loss: 0.3190 - learning_rate: 0.0010
Epoch 3/50
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 36s 3ms/step - accuracy: 0.8629 - loss: 0.3553 - val_accuracy: 0.8593 - val_loss: 0.3414 - learning_rate: 0.0010
Epoch 4/50
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 36s 3ms/step - accuracy: 0.8639 - loss: 0.3586 - val_accuracy: 0.8702 - val_loss: 0.3238 - learning_rate: 0.0010
Epoch 5/50
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 36s 3ms/step - accuracy: 0.8637 - loss: 0.3555 - val_accuracy: 0.8535 - val_loss: 0.4104 - learning_rate: 0.0010
Epoch 6/50
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 36s 3ms/step - accuracy: 0.8615 - loss: 0.3610 - val_accuracy: 0.8503 - val_loss: 0.3736 - learning_rate: 0.0010
Epoch 7/50
12353/12353 ━━━━━━━━━━━━━━━

In [19]:
best_model.save('modelo_final.keras')
# Guardar información de preprocesamiento
import pickle
preprocessing_info = {
    'feature_columns': feature_columns,
    'encoders': encoders,
    'scaler': scaler,
    'categorical_columns': existing_categorical
    }
    
with open('preprocessing_info.pkl', 'wb') as f:
    pickle.dump(preprocessing_info, f)

In [20]:
def test_predictions_on_sample(model_path, preprocessing_path, sample_data):
    """
    Probar predicciones en datos de muestra
    """
    model = tf.keras.models.load_model(model_path)
    
    with open(preprocessing_path, 'rb') as f:
        prep_info = pickle.load(f)
    
    # Procesar datos
    sample_processed = sample_data.copy()
    
    for col in prep_info['categorical_columns']:
        if col in sample_processed.columns:
            sample_processed[col] = sample_processed[col].fillna('unknown')
            le = prep_info['encoders'][col]
            # Manejar valores no vistos
            sample_processed[col] = sample_processed[col].astype(str).apply(
                lambda x: x if x in le.classes_ else le.classes_[0]
            )
            sample_processed[col] = le.transform(sample_processed[col])
    
    # Normalizar
    sample_scaled = prep_info['scaler'].transform(sample_processed[prep_info['feature_columns']])
    
    # Predecir
    probs = model.predict(sample_scaled, verbose=0)
    preds = (probs > 0.5).astype(int)
    
    return probs, preds

In [23]:
sample_data = df[existing_categorical].iloc[:100]
    
try:
    probs, preds = test_predictions_on_sample(
        'modelo_final.keras',
        'preprocessing_info.pkl',
        sample_data
        )
        
    print("Predicciones en muestra:")
    for i, (prob, pred) in enumerate(zip(probs.flatten(), preds.flatten())):
        real = df['resultado'].iloc[i]
        pred_label = 'exitoso' if pred == 1 else 'no_exitoso'
        print(f"  Ejemplo {i+1}: {pred_label} (prob: {prob:.4f}), Real: {real}")
            
except Exception as e:
    print(f"Error al probar predicciones: {e}")

Predicciones en muestra:
  Ejemplo 1: no_exitoso (prob: 0.0648), Real: no_exitoso
  Ejemplo 2: no_exitoso (prob: 0.0215), Real: no_exitoso
  Ejemplo 3: no_exitoso (prob: 0.2272), Real: no_exitoso
  Ejemplo 4: no_exitoso (prob: 0.1194), Real: no_exitoso
  Ejemplo 5: no_exitoso (prob: 0.0946), Real: no_exitoso
  Ejemplo 6: no_exitoso (prob: 0.2826), Real: no_exitoso
  Ejemplo 7: no_exitoso (prob: 0.1929), Real: no_exitoso
  Ejemplo 8: no_exitoso (prob: 0.0207), Real: no_exitoso
  Ejemplo 9: no_exitoso (prob: 0.0086), Real: no_exitoso
  Ejemplo 10: no_exitoso (prob: 0.1075), Real: no_exitoso
  Ejemplo 11: no_exitoso (prob: 0.0141), Real: no_exitoso
  Ejemplo 12: no_exitoso (prob: 0.0397), Real: no_exitoso
  Ejemplo 13: no_exitoso (prob: 0.0143), Real: no_exitoso
  Ejemplo 14: no_exitoso (prob: 0.0522), Real: no_exitoso
  Ejemplo 15: exitoso (prob: 0.5587), Real: no_exitoso
  Ejemplo 16: exitoso (prob: 0.8392), Real: exitoso
  Ejemplo 17: no_exitoso (prob: 0.1014), Real: no_exitoso
  Ejemp

In [31]:
df[categorical_columns+['resultado']].iloc[:30]

,cole_depto_ubicacion,cole_jornada,cole_naturaleza,fami_educacionmadre,fami_educacionpadre,fami_estratovivienda,resultado
0,CUNDINAMARCA,MAÑANA,OFICIAL,Primaria completa,Primaria completa,Estrato 2,no_exitoso
1,CAUCA,MAÑANA,OFICIAL,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,Estrato 1,no_exitoso
2,BOGOTA,MAÑANA,OFICIAL,Secundaria (Bachillerato) completa,Educación profesional completa,Estrato 2,no_exitoso
3,ANTIOQUIA,MAÑANA,OFICIAL,Secundaria (Bachillerato) incompleta,Ninguno,Estrato 2,no_exitoso
4,VALLE,MAÑANA,OFICIAL,Primaria incompleta,Primaria incompleta,Estrato 2,no_exitoso
5,BOGOTA,COMPLETA,NO OFICIAL,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,Estrato 2,no_exitoso
6,VALLE,MAÑANA,OFICIAL,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,Estrato 2,no_exitoso
7,QUINDIO,MAÑANA,OFICIAL,Secundaria (Bachillerato) completa,Primaria incompleta,Estrato 1,no_exitoso
8,ANTIOQUIA,TARDE,OFICIAL,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,Estrato 1,no_exitoso
9,CUNDINAMARCA,MAÑANA,OFICIAL,Secundaria (Bachillerato) incompleta,Primaria incompleta,Estrato 2,no_exitoso
